In [1]:
import numpy as np
x = np.random.rand(10, 1, 28, 28)
x.shape

(10, 1, 28, 28)

In [2]:
x[0,0,0]

array([0.29683804, 0.96811895, 0.2735978 , 0.22963474, 0.70879597,
       0.68802015, 0.55213554, 0.73910675, 0.60209161, 0.90443059,
       0.94851742, 0.06258858, 0.39054614, 0.68174967, 0.98853552,
       0.08988709, 0.07745115, 0.91393186, 0.64433122, 0.61843161,
       0.96711811, 0.73384663, 0.60710327, 0.8676216 , 0.74050354,
       0.58803321, 0.0788543 , 0.25400247])

- 如果老老实实地实现卷积运算，估计要重复好几层的for语句。这样的实现有点麻烦，而且，NumPy中存在使用for语句后处理变慢的缺点（NumPy中，访问元素时最好不要用for语句）。这里，我们不使用for语句，而是使用im2col这个便利的函数进行简单的实现。
- im2col会把输入的数据展开以适合滤波器。将应用滤波器的区域横向展开为1列。
- 使用im2col展开输入数据后，之后就只需将卷积层的滤波器（权重）纵向展开为1列，并计算2个矩阵的乘积即可
![](../pics/im2col.PNG)
卷积运算的滤波器处理的细节：将滤波器纵向展开为1列，并计算和im2col展开的数据的矩阵乘积，最后转换（reshape）为输出数据的大小

im2col (input_data, filter_h, filter_w, stride=1, pad=0)
- input_data―由（数据量，通道，高，长）的4维数组构成的输入数据
- filter_h―滤波器的高
- filter_w―滤波器的长
- stride―步幅
- pad―填充

In [5]:
import sys
import os
sys.path.append(os.pardir)
from common.util import im2col
x1 = np.random.rand(1, 3, 7, 7)
col1 = im2col(x1, 5, 5, 1, 0)
print(np.random.rand(3,4))
print(col1.shape)

x2=np.random.rand(10,3,7,7)
col2=im2col(x2,5,5,stride=1,pad=0)
print(col2.shape)

[[0.32571403 0.84109823 0.84132115 0.47065892]
 [0.465785   0.15969123 0.06789626 0.39717454]
 [0.93791833 0.88111311 0.95789614 0.36436248]]
(9, 75)
(90, 75)


In [4]:
class Convolution:
    def __init__(self, W, b, stride=1, pad=0) -> None:
        self.W = W
        self.b = b
        self.stride = stride
        self.pad = pad

    def forward(self, x):
        FN, C, FH, FW = self.W.shape
        N, C, H, W = x.shape
        # out_h的计算是上面写过的公式输出的高度为(1+H+2*P-FH)/S
        out_h = int(1+(H+self.pad*2-FH)/self.stride)
        out_w = int(1+(W+self.pad*2-FW)/self.stride)
        col = im2col(x, FH, FW, self.stride, self.pad)
        '''
        这里通过reshape(FN,-1)将参数指定为-1，这是reshape的一个便利的功能。
        通过在reshape时指定为-1，reshape函数会自动计算-1维度上的元素个数，
        以使多维数组的元素个数前后一致。
        比如，(10, 3, 5, 5)形状的数组的元素个数共有750个，
        指定reshape(10,-1)后，就会转换成(10, 75)形状的数组。
        '''
        col_W = self.W.reshape(FN, -1).T
        out = np.dot(col, col_W)+self.b
        out = out.reshape(N, out_h, out_w, -1).transpose(0, 3, 1, 2)
        return out


forward的实现中，最后会将输出大小转换为合适的形状。转换时使用了NumPy的transpose函数。transpose会更改多维数组的轴的顺序。如图7-20所示，通过指定从0开始的索引（编号）序列，就可以更改轴的顺序。
![](../pics/transpose_function.PNG)

In [3]:
class Pooling:
    def __init__(self, pool_h, pool_w, stride=1, pad=0) -> None:
        self.pool_h = pool_h
        self.pool_w = pool_w
        self.stride = stride
        self.pad = pad

    def forward(self, x):
        N, C, H, W = x.shape
        out_h = int(1+(H-self.pool_h)/self.stride)
        out_w = int(1+(W-self.pool_w)/self.stride)
        col = im2col(x, self.pool_h, self.pool_w, self.stride, self.pad)
        col = col.reshape(-1, self.pool_h*self.pool_w)
        out = np.max(col, axis=1)
        out = out.reshape(N, out_h, out_w, C).transpose(0, 3, 1, 2)
        return out

input_d=(1,28,28)

type(input_d)

tuple

28